<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Marvel API Integration with SQLite and Pandas**

In [1]:
import hashlib
import requests
import pandas as pd
import sqlite3
from google.colab import userdata

# Obtém as chaves de API do usuário.
def get_api_keys():
    return userdata.get('PUBLIC_KEY'), userdata.get('PRIVATE_KEY')

# Gera o hash MD5 necessário para autenticação na API / Timestamp fixo.
def generate_md5_hash(public_key, private_key):
    ts = "1"
    hash_input = f"{ts}{private_key}{public_key}"
    return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

### ***Faz uma requisição GET à API e retorna os dados de todas as páginas em formato JSON.***

In [2]:
def fetch_data_from_api(endpoint, public_key, hash_md5):
    base_url = "http://gateway.marvel.com"
    url = f"{base_url}/{endpoint}?ts=1&apikey={public_key}&hash={hash_md5}&limit=100"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

### **Cria as tabelas no banco de dados, caso não existam.**

In [3]:
def create_tables(cursor):
    cursor.execute("""CREATE TABLE IF NOT EXISTS characters (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        description TEXT,
                        comics_returned INTEGER,
                        stories_returned INTEGER,
                        events_returned INTEGER)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS comics (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        comic_title TEXT,
                        characters_returned INTEGER,
                        events_returned INTEGER)""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS events (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        description TEXT,
                        start DATE,
                        end DATE,
                        characters_returned INTEGER,
                        comics_returned INTEGER)""")

### **Insere os dados na tabela especificada do banco de dados.**

In [4]:
def insert_data_into_db(cursor, data, table_name):
    for item in data:
        if table_name == 'characters':
            # Inserindo os dados na tabela 'characters'
            cursor.execute("""INSERT OR IGNORE INTO characters (id, name, description, comics_returned, stories_returned, events_returned)
                              VALUES (?, ?, ?, ?, ?, ?)""",
                              (item["id"], item["name"], item["description"],
                               item["comics"]['returned'], item["stories"]['returned'], item["events"]['returned']))
        elif table_name == 'comics':
          # Inserindo os dados na tabela 'comics'
          cursor.execute("""INSERT OR IGNORE INTO comics ( comic_title, characters_returned, events_returned)
                         VALUES ( ?, ?, ?)""", ( item["title"], item["characters"]["returned"] , item["events"]["returned"]))
        elif table_name == 'events':
           # Inserindo os dados na tabela 'events'
           cursor.execute("""INSERT OR IGNORE INTO events (title, description, start, end, characters_returned, comics_returned )
                              VALUES (?, ?, ?, ?, ?, ?)""", (item["title"], item["description"], item["start"], item["end"], item["characters"]["returned"], item["comics"]['returned']))

### **Função principal que executa todas as etapas do processo.**

In [ ]:
def main():
    try:
        public_key, private_key = get_api_keys()
        hash_md5 = generate_md5_hash(public_key, private_key)

        # Fetching data from Marvel API
        characters_data = fetch_data_from_api("v1/public/characters", public_key, hash_md5)["data"]["results"]
        comics_data = fetch_data_from_api("v1/public/comics", public_key, hash_md5)["data"]["results"]
        events_data = fetch_data_from_api("v1/public/events", public_key, hash_md5)["data"]["results"]

        # Conectando ao banco de dados SQLite
        with sqlite3.connect('marvel.db') as conn:
            cursor = conn.cursor()

            # Criando as tabelas no banco de dados
            create_tables(cursor)

            # Inserindo os dados nas tabelas
            insert_data_into_db(cursor, characters_data, 'characters')
            insert_data_into_db(cursor, comics_data, 'comics')
            insert_data_into_db(cursor,events_data, 'events')

        # Lendo a tabela de personagens com pandas
        with sqlite3.connect('marvel.db') as conn:
            personagens_df = pd.read_sql_query("SELECT * FROM characters", conn)
            comics_df = pd.read_sql_query("SELECT * FROM comics", conn)
            events_df = pd.read_sql_query("SELECT * FROM events", conn)


        # Carrega os dados da tabela personagem como dataframe do Pandas
        df_Personagem = pd.DataFrame(personagens_df, columns = ['id', 'name', 'description', 'comics_returned', 'stories_returned', 'events_returned'])
        df_comics = pd.DataFrame(comics_df, columns=['id','comic_title','characters_returned','events_returned'])
        df_events = pd.DataFrame(events_df, columns=['id','title','description','start','end','characters_returned','comics_returned'])

        print("Tabela de Personagem")
        display(df_Personagem)
        print("Tabela de Comics")
        display(df_comics)
        print("Tabela de Events")
        display(df_events)

        #Carrega os dados em um arquivo .csv
        df_Personagem.to_csv("df_Personagem.csv",encoding="utf-8",index=False,header=True,sep=";")
        df_comics.to_csv("df_comics.csv",encoding="utf-8",index=False,header=True,sep=";")
        df_events.to_csv("df_events.csv",encoding="utf-8",index=False,header=True,sep=";")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição à API: {e}")
    except sqlite3.DatabaseError as e:
        print(f"Erro no banco de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")

main()